In [1]:
import numpy as np
np.set_printoptions(threshold=np.nan)

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler

In [3]:
X = load_breast_cancer().data
y = load_breast_cancer().target

Note that the tensorflow code below did not converge until the data were standardized.

In [4]:
X_std = StandardScaler().fit_transform(X)

In [5]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
print lr.fit(X_std, y).score(X_std, y)

0.98769771529


In [6]:
lr.coef_

array([[-0.35372245, -0.3850941 , -0.34237238, -0.44138446, -0.15523716,
         0.5681635 , -0.8685186 , -0.96811443,  0.07328189,  0.31122062,
        -1.29527365,  0.26995006, -0.6662383 , -1.02954508, -0.2812678 ,
         0.74241788,  0.11352258, -0.32006685,  0.28982672,  0.67152689,
        -1.0304876 , -1.3131883 , -0.82563973, -1.02915516, -0.67185301,
         0.04896119, -0.87162239, -0.91131563, -0.8839543 , -0.48354624]])

In [7]:
lr.intercept_

array([ 0.17993455])

In [8]:
y_pred_prob = lr.fit(X_std, y).predict_proba(X_std)
loss = -(np.log(y_pred_prob[y == 0][:,0]).sum() + np.log(y_pred_prob[y == 1][:,1]).sum()) / y_pred_prob.size
loss

0.026659267587480589

In [9]:
import tensorflow as tf

In [10]:
tf.reset_default_graph()

In [11]:
X = tf.constant(X_std)
y = tf.constant(y.reshape(-1, 1), dtype=tf.int32)

In [12]:
k_init = tf.truncated_normal_initializer(mean=0.0, stddev=1.0, seed=42)
prob_positive = tf.layers.dense(X, units=1, activation=tf.sigmoid, kernel_initializer=k_init, name='single_neuron')
loss = tf.losses.log_loss(labels=y, predictions=prob_positive)

In [13]:
optimizer = tf.train.MomentumOptimizer(learning_rate=1.0, momentum=0.975, use_nesterov=True)
training_op = optimizer.minimize(loss)

In [14]:
accuracy, accuracy_op = tf.metrics.accuracy(labels=y, predictions=tf.round(prob_positive))
accuracy2, accuracy_op2 = tf.metrics.accuracy(labels=y, predictions=tf.round(prob_positive))
acc = tf.reduce_sum(tf.cast(tf.equal(y, tf.cast(prob_positive + 0.5, tf.int32)), tf.int32))
acc2 = tf.reduce_sum(tf.cast(tf.equal(y, tf.cast(tf.round(prob_positive), tf.int32)), tf.int32))
numm = tf.cast(tf.size(y), tf.float32)
jj = tf.cast(prob_positive + 0.5, tf.int32)
hh = tf.cast(tf.round(prob_positive), tf.int32)
vv = tf.cast(prob_positive > 0.5, tf.int32)

In [15]:
init = tf.global_variables_initializer()
lcl = tf.local_variables_initializer()

In [16]:
n_epochs = 5000

In [17]:
with tf.Session() as sess:
     init.run()
     lcl.run()
     sess.run(training_op)
     #print np.c_[prob_positive.eval(), jj.eval(), hh.eval(), vv.eval()]
     print prob_positive.eval().min(), prob_positive.eval().max()
     for epoch in xrange(n_epochs + 1): 
          sess.run(training_op)
          if not (epoch % 100):
               print "Loss:", loss.eval(), "Acc:", accuracy.eval(), accuracy_op.eval(), acc.eval() / float(X_std.shape[0]), acc2.eval() / float(X_std.shape[0])
     if not (epoch % 100): print "Loss:", loss.eval(), "Acc:", accuracy2.eval(), accuracy_op2.eval()

     print tf.get_default_graph().get_tensor_by_name('single_neuron/kernel:0').eval()
     print tf.get_default_graph().get_tensor_by_name('single_neuron/bias:0').eval()
     print prob_positive.eval().min(), prob_positive.eval().max()

4.50177345016e-19 0.999999999999
Loss: 0.225255 Acc: 0.0 0.947276 0.947275922671 0.947275922671
Loss: 0.155675 Acc: 0.947276 0.967487 0.98769771529 0.98769771529
Loss: 0.14224 Acc: 0.967487 0.975395 0.991212653779 0.991212653779
Loss: 0.14183 Acc: 0.975395 0.97935 0.991212653779 0.991212653779
Loss: 0.141777 Acc: 0.97935 0.981722 0.991212653779 0.991212653779
Loss: 0.141752 Acc: 0.981722 0.983304 0.991212653779 0.991212653779
Loss: 0.141736 Acc: 0.983304 0.984434 0.991212653779 0.991212653779
Loss: 0.141725 Acc: 0.984434 0.985281 0.991212653779 0.991212653779
Loss: 0.141716 Acc: 0.985281 0.98594 0.991212653779 0.991212653779
Loss: 0.141708 Acc: 0.98594 0.986467 0.991212653779 0.991212653779
Loss: 0.141702 Acc: 0.986467 0.986899 0.991212653779 0.991212653779
Loss: 0.141697 Acc: 0.986899 0.987258 0.991212653779 0.991212653779
Loss: 0.141693 Acc: 0.987258 0.987563 0.991212653779 0.991212653779
Loss: 0.141689 Acc: 0.987563 0.987823 0.991212653779 0.991212653779
Loss: 0.141686 Acc: 0.987823